# Level 1 and Level 2 Landsat 8 data:
## A comparison using Modified Normalised Difference Water Index (MNDWI)

* **Products used:** 
[ls8_usgs_sr_scene](https://explorer.digitalearth.africa/ls8_usgs_sr_scene)
* **Level 1 data retrived from:**
[Landsat 8 Level 1 data](https://registry.opendata.aws/landsat-8/)

## Background
### Analysis-Ready Data (ARD): the difference between Level 1 and Level 2 data

Satellite operators, such as USGS, perform minimal processing on the data before releasing it for general consumption. This is known as **Level 1 data**. As Level 1 data is very close to its raw form, it can be flexibly manipulated by the scientific community, but requires specialist knowledge to use. Atmospheric interference and other underlying satellite data issues must be corrected for before the data shows meaningful results.

The accessibility of available satellite data can therefore be significantly increased by further processing the Level 1 data, such as by applying atmospheric corrections. This processed data is hosted on the OpenDataCube, and is known as **Level 2 data** or **Analysis-Ready Data**, shortened to **ARD**. The corrections improve the overall quality of the data (compared to the Level 1 data) and allow users to conduct analysis independent of expertise in the data corrections process.

### Modified Normalised Difference Water Index (MNDWI)

MNDWI is a variation on the Normalised Difference Water Index (NDWI), which is calculated using the Green band and Near-Infrared (NIR) bands from satellite data.

\begin{equation} \text{NDWI} \ = \ \frac{\text{Green} - \text{NIR}}{\text{Green} + \text{NIR}} \end{equation}

However, it has been shown that using a Short-Range Infrared (SWIR) band instead of NIR provides enhanced delineation between water and other types of areas (soil, vegetation, built-up land). This gives the Modified NDWI, or MNDWI.

\begin{equation} \text{MNDWI} \ = \ \frac{\text{Green} - \text{SWIR}}{\text{Green} + \text{SWIR}} \end{equation}

In the case of Landsat 8 satellite data, which has two SWIR bands, SWIR 1 is used. Note the indices are normalised, which means their values range from -1 to 1. Practically, this can be interpreted as:

* MNDWI $> 0$ : water
* MNDWI $\leq 0$ : soil, vegetation, urban areas

## Description

This notebook will show the ARD (Level 2) data produces better results than uncorrected (Level 1) data. This is done by calculating MNDWI using Level 1 data and Level 2 data from Landsat 8, for the same region. The results show the ARD data shows clearer signals and more distinct patterns.

The notebook contains the following steps:
1. Fulfill prerequisites: Select a location for analysis and download Level 1 data.
2. Load Level 1 data for the location.
3. Load Level 2 data for the location.
4. Calculate MNDWI for the Level 1 data.
5. Calculate MNDWI for the Level 2 data.
6. Compare the Level 1 and Level 2 data by plotting them together.

*****

## Getting Started
To run this analysis, ensure the prerequesites are fulfilled, then run all the cells in the notebook, starting with the "Load packages" cell. 

### Prerequisites

* Choosing location

The Digital Earth Africa sandbox does not contain a product for Landsat 8 Level 1 data. This means Level 1 data must be manually downloaded from the USGS website before it can be imported into the notebook. A time and place for analysis must be decided.

For the purposes of this notebook, an area off the coast of Guinea Bissau, Senegal, has been selected.

`lat = 11.228`<br>
`lon = -15.860`

* Choosing time

Here, MNDWI is calculated for a single instance in time. The time chosen for this notebook was `January 2018`.

* Downloading Level 1 data

*Note: this requires an active Amazon Web Services (AWS) account.*

The time and place can now be searched on the [DE Africa Metadata Explorer](https://explorer.digitalearth.africa/ls8_usgs_sr_scene). If not preselected, the selected product should be `ls8_usgs_sr_scene`. Use the `Year` and `Month` dropdowns to select `2018` and `January` as appropriate.

By browsing the `Day` dropdown, the only scene encapsulating the Bissau coast is `9 January 2018`. Click `11 datasets available` on the right-hand toolbar to open details of each scene taken on that day. The Bissau coast is the last dataset; [13b9c5f0-65f0-5d2e-8473-5bbfc92a9673](https://explorer.digitalearth.africa/dataset/13b9c5f0-65f0-5d2e-8473-5bbfc92a9673). Note the `Locations > Active:` field as it shows the naming convention for this scene.

Navigate to the [USGS Landsat 8 registry](https://registry.opendata.aws/landsat-8/) and access their AWS S3 Bucket. The file naming convention is the same as in the DE Africa explorer; i.e. look for files starting with `LC08_L1TP_204052_20180109_20180119_01_T1`. 

The folder for this scene will contain many files. Download the Band 3 `_B3` and Band 6 `_B6` `.tiff` files and upload them to the same Jupyterlab folder as this notebook.


### Load packages
Import Python packages that are used for the analysis.

In [1]:
%matplotlib inline

import datacube
from datacube.testutils import io
import sys
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np

sys.path.append("../Scripts")
from deafrica_bandindices import calculate_indices